In [1]:
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import random
import pandas as pd
import pickle
import csv, io, requests

In [2]:
import nltk

In [3]:
df_users = pd.read_pickle('C:/Users/Julia/Documents/CompTools/users_typecasted.pkl')
df_users = df_users.loc[df_users['Id'] > 0]

df_users.head()

,Id,Reputation,CreationDate,LastAccessDate,Views,UpVotes,DownVotes
16,1,63320,2008-07-31 14:22:31.287,2023-07-03 03:32:33.740,594722,3407,1312
19,4,33372,2008-07-31 14:22:31.317,2023-07-11 19:52:52.090,84130,855,100
20,5,52327,2008-07-31 14:22:31.317,2023-03-07 02:39:34.550,15166,790,34
22,9,25430,2008-07-31 21:35:26.517,2023-09-01 23:18:04.763,7713,47,4
25,13,219335,2008-08-01 04:18:04.943,2022-07-28 18:13:37.620,45326,5212,210


In [4]:
df_comments_1 = pd.read_pickle('C:/Users/Julia/Downloads/comments_typecasted_1.pkl')
df_comments_2 = pd.read_pickle('C:/Users/Julia/Downloads/comments_typecasted_2.pkl')
df_comments_3 = pd.read_pickle('C:/Users/Julia/Downloads/comments_typecasted_3.pkl')
df_comments_4 = pd.read_pickle('C:/Users/Julia/Downloads/comments_typecasted_4.pkl')
df_comments_5 = pd.read_pickle('C:/Users/Julia/Downloads/comments_typecasted_5.pkl')

In [5]:
df_comments = pd.concat([df_comments_1, df_comments_2, df_comments_3, df_comments_4, df_comments_5])

In [6]:
df_comments['PostId'] = pd.to_numeric(df_comments['PostId'])

In [7]:
df_posts_1 = pd.read_pickle('C:/Users/Julia/Downloads/posts_typecasted_1.pkl')
df_posts_2 = pd.read_pickle('C:/Users/Julia/Downloads/posts_typecasted_2.pkl')
df_posts_3 = pd.read_pickle('C:/Users/Julia/Downloads/posts_typecasted_3.pkl')
df_posts_4 = pd.read_pickle('C:/Users/Julia/Downloads/posts_typecasted_4.pkl')

In [8]:
df_posts = pd.concat([df_posts_1,df_posts_2,df_posts_3,df_posts_4])

In [9]:
df_posts['OwnerUserId'] = pd.to_numeric(df_posts['OwnerUserId'])
df_answers = df_posts.loc[df_posts['PostTypeId'] == 2]
df_answers.head()

,Id,PostTypeId,ParentId,AcceptedAnswerId,CreationDate,Score,ViewCount,Body,OwnerUserId,LastActivityDate,Title,Tags,AnswerCount,CommentCount
150,307,2,260,-1,2008-08-02 01:49:46.220,43,-1,"<p><a href=""https://www.codeproject.com/Articl...",49,2019-12-24 07:28:19.907,Comment: N/A,Comment: N/A,-1,3
162,342,2,337,-1,2008-08-02 04:01:34.600,36,-1,"<p>Personally, I've played with several of the...",59,2008-08-02 04:01:34.600,Comment: N/A,Comment: N/A,-1,1
197,471,2,337,-1,2008-08-02 15:21:03.587,33,-1,"<p><a href=""http://effbot.org/zone/element-ind...",147,2008-08-02 15:21:03.587,Comment: N/A,Comment: N/A,-1,2
210,525,2,337,-1,2008-08-02 18:04:10.543,6,-1,<p>Since you mentioned that you'll be building...,154,2008-08-02 18:04:10.543,Comment: N/A,Comment: N/A,-1,0
218,538,2,535,-1,2008-08-02 18:56:56.460,29,-1,<p>One possibility is Hudson. It's written in...,156,2013-01-28 03:54:17.217,Comment: N/A,Comment: N/A,-1,4


In [10]:
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Julia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Julia\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Julia\AppData\Roaming\nltk_data...


True

In [40]:
# '''Following code combined stopwords from 3 sources and gives a stopword list as an output'''
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction import _stop_words # from sklearn
from nltk.corpus import stopwords  # from nltk
import string
from nltk import *
''' Creates a list of stop words by combining three stop word libraries and a custom list'''
def combined_stop_words_list():
    list_stop_words= [word for word in _stop_words.ENGLISH_STOP_WORDS]

    nltk_stopwords = [word for word in stopwords.words('english')]
    list_stop_words = list_stop_words+nltk_stopwords

    list_stop_words=set(list_stop_words)

    added_words = []
    list_stop_words = [word for word in list_stop_words]+added_words
    list_stop_words = set(list_stop_words)
    print("Stopword list generation complete")
    return list_stop_words
list_stop_words = combined_stop_words_list()



'''
This code performs the following operations:
    - removes alphanumeric characters
    - removes multiple whitespaces
    - removes stopwords before converting to lowercase
    - removes repeating letter sequences such as aaa or zzz
    - conversion to lowercase
    - performs lemmatization
    - remove stopwords
    - remove punctuation
'''
def clean_up(text):
    
   
    text = "".join([c.lower() for c in text])  # lowercase       
    text = " ".join([c for c in text.split() if len(c) > c.count(c[0])]) #remove words with repeating letter sequences or single digits
    text = re.sub(r'''
    \W+       # Accept one or more non-word characters
    \s*       # plus zero or more whitespace characters,
    ''',      # Close string block
    ' ',      # and replace it with a single space
    text,
    flags=re.VERBOSE)
    text = re.sub(r'\w*\d\w*', '', text)   
    text = " ".join([w for w in text.split() if w not in list_stop_words])   
    lemmatizer = WordNetLemmatizer()
    text = " ".join([lemmatizer.lemmatize(w) for w in text.split()]) 
    text = "".join([c for c in text if c not in string.punctuation])
    text = re.sub(r'\b\w{1,2}\b', '', text)
    text = " ".join([w for w in text.split() if w not in list_stop_words])
    return text

Stopword list generation complete


In [58]:
reader = csv.reader(io.StringIO(requests.get('https://journals.plos.org/plosone/article/file?type=supplementary&id=info:doi/10.1371/journal.pone.0026752.s001').text), delimiter='\t')
word_list = [word for word in reader] # Couldn't call list(reader) due to previous variable assignment
sentiment_dictionary = {}
for word in word_list[4:]: #removing the headers
    sentiment_dictionary[word[0]] = float(word[2]) #iterating over each word and adding it with happines_average

'''
Calculate sentiment for a set of tokens
'''
def sentiment(tokens):
    sent = np.sum([sentiment_dictionary[word] for word in tokens if word in sentiment_dictionary.keys()])
    return sent if sent is not None else 0

def tokens_from_comment(comment):
    return clean_up(comment).split()

def get_sentiment_for_answer(answerId):
    list_of_comments = df_comments.loc[df_comments['PostId'] == answerId]['Text'].tolist()
    return np.sum([sentiment(tokens_from_comment(comment)) for comment in list_of_comments])

def get_sentiment_for_user(userId):
    user_answer_ids = df_answers.loc[df_answers['OwnerUserId'] == userId]['Id'].tolist()
    if len(user_answer_ids) == 0:
        return None
    return np.mean([get_sentiment_for_answer(ans) for ans in user_answer_ids])
    

In [59]:
users_ids_with_answers = set(df_answers['OwnerUserId'].unique())

df_users_w_answers = df_users[df_users['Id'].isin(users_ids_with_answers)]

use_sent = df_users_w_answers.head(1000)
use_sent['AnswersSentiment'] = use_sent['Id'].apply(lambda x: get_sentiment_for_user(x))

C:\Users\Julia\AppData\Local\Temp\ipykernel_25016\4082889836.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  use_sent['AnswersSentiment'] = use_sent['Id'].apply(lambda x: get_sentiment_for_user(x))


In [60]:
use_sent

,Id,Reputation,CreationDate,LastAccessDate,Views,UpVotes,DownVotes,AnswersSentiment
16,1,63320,2008-07-31 14:22:31.287,2023-07-03 03:32:33.740,594722,3407,1312,0.000000
19,4,33372,2008-07-31 14:22:31.317,2023-07-11 19:52:52.090,84130,855,100,60.920000
20,5,52327,2008-07-31 14:22:31.317,2023-03-07 02:39:34.550,15166,790,34,0.000000
22,9,25430,2008-07-31 21:35:26.517,2023-09-01 23:18:04.763,7713,47,4,0.000000
25,13,219335,2008-08-01 04:18:04.943,2022-07-28 18:13:37.620,45326,5212,210,172.236000
...,...,...,...,...,...,...,...,...
3282,4834,37849,2008-09-05 20:27:56.450,2023-09-03 11:38:32.920,5882,5196,198,55.095033
3288,4842,1976,2008-09-05 22:23:54.877,2017-10-09 02:31:55.180,388,137,0,16.040000
3293,4849,875,2008-09-06 00:06:08.967,2015-07-30 23:14:41.547,61,4,0,0.000000
3294,4850,13788,2008-09-06 00:59:14.127,2023-09-02 19:01:48.080,1233,447,58,66.530000


In [63]:
np.max(use_sent['AnswersSentiment'])

709.69